# Quest: Repo Ranger

    - Goal: Label pull requests via deterministic rules
- Goal: Route risky changes through workflow

    **Pass criteria:** PR report saved with labels and workflow routes.

    _Mock mode uses fixtures and writes artifacts under `_runs/`. Flip `USE_REAL_CLIENT=True` when ready for live calls._

In [ ]:

USE_REAL_CLIENT = False  # flip when ready
import os, json, time, pathlib, math, random

RUN = pathlib.Path('_runs/quest_repo_ranger')
RUN.mkdir(parents=True, exist_ok=True)
FIXTURE_ROOT = pathlib.Path(os.environ.get('MOCK_FIXTURE_ROOT', '_fixtures'))

def write_meta(**kwargs):
    payload = {"timestamp": time.time(), **kwargs}
    (RUN / 'meta.json').write_text(json.dumps(payload, indent=2))
    return payload

USE_REAL_CLIENT = bool(str(os.environ.get('USE_REAL_CLIENT', USE_REAL_CLIENT)).lower() in {'1', 'true', 'yes'})


In [ ]:
pull_requests = [
    {"id": 101, "files": ["notebooks/01_streaming_basics.ipynb"], "tests": True, "lines_changed": 42},
    {"id": 102, "files": ["tools/backoff.py", "core/runtime.py"], "tests": False, "lines_changed": 215},
    {"id": 103, "files": ["docs/index.md"], "tests": False, "lines_changed": 12},
]
reports = []
for pr in pull_requests:
    labels = []
    if any(path.startswith("docs") for path in pr["files"]):
        labels.append("docs")
    if pr["lines_changed"] > 200 or not pr["tests"]:
        labels.append("needs-review")
    if any(path.startswith("tools") for path in pr["files"]):
        labels.append("runtime")
    requires_workflow = "needs-review" in labels
    reports.append({**pr, "labels": labels, "requires_workflow": requires_workflow})
(RUN / "pr_reports.json").write_text(json.dumps(reports, indent=2))
reports

In [ ]:
meta = write_meta(ok=True, lab="quest_repo_ranger", risky=sum(1 for pr in reports if pr['requires_workflow']))
meta